In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [2]:
import pickle
import requests
import json

In [3]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import stopwords
from subprocess import check_output
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix

In [5]:
#models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [6]:
from textblob.classifiers import NaiveBayesClassifier

In [7]:
from sklearn.neural_network import MLPClassifier

In [8]:
emot = pd.read_csv("ISEAR.csv",encoding='latin1')
emot.sample()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,SELF,RELA,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE
1161,41042,1,4,42,2,24,8,2,5,5,...,2,2,2,0,anger,4,4,410423,The outrageous conduct of a relative towards m...,1


In [9]:
emot = emot[["SIT","Field1","SEX","SUBJ"]]
emot.SIT[0]

'During the period of falling in love, each time that we met and á\nespecially when we had not met for a long time.'

In [10]:
emot.Field1.value_counts()

shame      1096
sadness    1096
anger      1096
disgust    1096
fear       1095
joy        1094
guilt      1093
Name: Field1, dtype: int64

In [11]:
lem = WordNetLemmatizer()
def clean_it (text) :
    text = ' '.join([lem.lemmatize(w) for w in text.split()])
    text = re.sub("n\'t","\snot\s",text)
    text = re.sub("\(.*?\)","",text)
    text = re.sub("\[.*?\]","",text)
    text = re.sub(r'\".*?\"',"",text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub(r'\b\w*\d+\w*\b','',text)
    text = re.sub(r"\b[^\s][^\s]\b","",text)
    text = re.sub(r"\b[^\s]\b","",text)
#     text = re.sub("^\w$","",text)
    text = re.sub("\d+","",text)
#     text = re.sub("^/w/w$","",text)
#     text = re.sub("^.?$","",text)
    text = re.sub('\n', '', text)
    text = re.sub('\á', '', text)

    return text
clean = lambda x : clean_it(x) 

In [12]:
cleaned = emot.SIT.apply(clean)

In [13]:
word_len = pd.Series(cleaned).apply(lambda x : len(x.split()))

In [14]:
emot["word_len"] = word_len

In [15]:
emot["SIT"] = cleaned

In [16]:
emot = emot[emot.word_len >= 17]

In [17]:
for i in cleaned:
    if "wa" in i :
        print("\n",i)


 when  saw all the very drunk kid   town   walpurgis night 

 when  realized that   directing the feeling  discontent  with myself   partner and this way  trying  put the blame   him instead  sorting out  own feeliings 

 when  recently thought about the hard work  take  study  and  how one want  try something else  when  read  theoretical  book  english that  did not understand 

 when one find out that someone you know  not  all like one  had thought  for instance friend who steal and thing like that   quite unwarranted 

 when one  been unjust  stupid towards someone else 

 when  see child    from area devastated  drought and  war 

 when  nearly walked   blindworm and then saw  crawl away 

   winter spring and the ice  melting    far out    jetty and  poked the ice with  long branch  try and break    the branch went through the ice and  fell   the water closed   over  head 

 unjust accusation directed   and  way  acting    someone close   

 when  heard  the radio that the foot

    march  when one   fellow student borrowed   wristwatch and after sometime  broke  glass  that watch   got angry when   telling  but the anger did snot  take long   fade 

 when   about  write  junior certificate exams   received  sad news telling  that  elder sister had passed away 

 when   told that  father had passed away 

 this situation occurred when  entered into  smelling ward 

  felt sad when   told  walk out  the classroom because    within the row  student who were making noise 

  girlfriend who had been very far away from  came    place  she saw  and ran after    did snot  know that she had  come   could snot  see her  she held  shoulder standing behind   

   given  wristwatch   first girl and after two day  another girl  mine also gave   wristwatch 

  had  girlfriend  the same class  was   did not  well    exams  but she did satisfactorily well and  taken into  another class leaving  behind 

  really became sad when mother passed away while   very  young 

  frien

In [18]:
set(stopwords.words("english"))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [19]:
custom_sw = list(set(stopwords.words("english")))
custom_sw.remove('not')

In [20]:
X_train,X_test,y_train,y_test = train_test_split(emot.SIT,emot.Field1,test_size = 0.2,random_state = 42)
vectorizer = TfidfVectorizer(stop_words = custom_sw)
X_train = vectorizer.fit_transform(X_train)

a = cleaned
tv = TfidfVectorizer()
a= tv.fit_transform(a)

# a.toarray()
print("\n")
for i in tv.get_feature_names() :
    if len(i) < 3:
        print(i)

In [21]:
from pprint import pprint

In [22]:
a = list(vectorizer.vocabulary_.keys())
a.sort()
pprint(a)

['abandoned',
 'abdomen',
 'abdominal',
 'abhorrent',
 'abilities',
 'ability',
 'able',
 'abortion',
 'abroad',
 'abrupt',
 'abruptly',
 'abscence',
 'absence',
 'absent',
 'absolutely',
 'abuse',
 'abused',
 'abusing',
 'abusive',
 'abut',
 'academic',
 'academically',
 'academics',
 'academy',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accident',
 'accidentally',
 'accidently',
 'accomodation',
 'accompanied',
 'accompanying',
 'accomplish',
 'accomplished',
 'according',
 'account',
 'accquaintances',
 'accquainted',
 'accquired',
 'accquiring',
 'accusations',
 'accused',
 'ache',
 'achieve',
 'achieved',
 'achievment',
 'acknowledge',
 'acquaintance',
 'acquaintanced',
 'acquaintances',
 'acquainted',
 'acquaintences',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'active',
 'activities',
 'activity',
 'actor',
 'actors',
 'actual',
 'actually',
 'adapt',
 'add',
 'added',
 'addiction',
 'addition',
 'additional',
 'additonal',
 'addres

 'guarusa',
 'guess',
 'guessed',
 'guest',
 'guidance',
 'guide',
 'guilt',
 'guiltier',
 'guilty',
 'guinea',
 'gule',
 'gulf',
 'gully',
 'gum',
 'gun',
 'gush',
 'gushing',
 'gutter',
 'guy',
 'gym',
 'gymnastics',
 'gypsy',
 'habit',
 'habitual',
 'habitually',
 'haded',
 'hah',
 'hair',
 'hairdresser',
 'hale',
 'half',
 'hall',
 'halloween',
 'hamburger',
 'hamburgers',
 'hampering',
 'hand',
 'handbag',
 'handed',
 'handicapped',
 'handing',
 'handkerchief',
 'handle',
 'handled',
 'handling',
 'hands',
 'hanging',
 'happen',
 'happened',
 'happening',
 'happens',
 'happiest',
 'happily',
 'happiness',
 'happy',
 'harassment',
 'hard',
 'harder',
 'hardly',
 'harm',
 'harmfully',
 'harming',
 'harmonic',
 'harrassed',
 'harrassing',
 'harsh',
 'harshly',
 'hassled',
 'hasty',
 'hate',
 'hated',
 'hatred',
 'haunted',
 'head',
 'headed',
 'headman',
 'headmaster',
 'headmistress',
 'health',
 'healthy',
 'heaped',
 'hear',
 'heard',
 'hearing',
 'hears',
 'heart',
 'heartedly',


 'rumphi',
 'run',
 'runaround',
 'running',
 'rush',
 'rushed',
 'rushing',
 'russians',
 'rustling',
 'sad',
 'saddened',
 'saddening',
 'saddned',
 'sadness',
 'safe',
 'safely',
 'safety',
 'sagged',
 'said',
 'sailing',
 'sailor',
 'sake',
 'salary',
 'sales',
 'salesperson',
 'saliva',
 'salo',
 'sam',
 'sametime',
 'sanatorium',
 'sand',
 'sang',
 'sanity',
 'sank',
 'sarcastic',
 'sat',
 'satanic',
 'satisfaction',
 'satisfactorily',
 'satisfactory',
 'satisfied',
 'satisfy',
 'saturday',
 'sauna',
 'save',
 'saved',
 'saving',
 'saw',
 'say',
 'saying',
 'scale',
 'scales',
 'scandal',
 'scar',
 'scare',
 'scared',
 'scaring',
 'scary',
 'scattered',
 'scenary',
 'scene',
 'scenery',
 'schhol',
 'scholarship',
 'school',
 'schoolmate',
 'schoolwork',
 'science',
 'scienced',
 'sciences',
 'scientists',
 'scissors',
 'scold',
 'scolded',
 'scolding',
 'scooter',
 'scope',
 'score',
 'scored',
 'scottish',
 'scoundrel',
 'scouting',
 'scraped',
 'scratch',
 'scream',
 'screamed'

In [23]:
y_train

7339       fear
4636       fear
3164      shame
1909      guilt
1023    sadness
6129      anger
5365      guilt
5971      shame
7305      anger
585        fear
2646      guilt
2409      shame
7581      guilt
4537        joy
1522       fear
7542      anger
6177       fear
4905    disgust
5815       fear
2325       fear
6135       fear
5556        joy
3677      guilt
5140      shame
4584      shame
5253      guilt
6464      anger
955       shame
2998      shame
6163       fear
         ...   
2740       fear
5737        joy
7333      anger
3066      shame
1508       fear
2157       fear
6330       fear
64         fear
1488      anger
3274        joy
6379       fear
2689      anger
3348      anger
6610      anger
4096      guilt
6541    sadness
2248       fear
4714      anger
422     disgust
5958      shame
4035      guilt
1177    disgust
3442      shame
1497    disgust
6038      anger
4580       fear
3093    disgust
3178      shame
3567      shame
2435    sadness
Name: Field1, Length: 22

In [24]:
X_test = vectorizer.transform(X_test)

In [25]:
# X_test.toarray()

In [26]:
# y_test = vectorizer.transform(y_test)

In [46]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
lr_predictions = model.predict(X_test)

In [48]:
accuracy_score(y_test,lr_predictions)

0.6371841155234657

In [66]:
trial = ['''could you be any happy''']
a = vectorizer.transform(trial)
print(model.predict(a)[0])
model.predict_proba(a)
print(list(pd.Series(trial).apply(clean)))

joy
['could you  any happy']


In [50]:
print(classification_report(y_test,lr_predictions))

              precision    recall  f1-score   support

       anger       0.50      0.66      0.57        92
     disgust       0.70      0.56      0.62        71
        fear       0.65      0.85      0.74       100
       guilt       0.55      0.54      0.54        80
         joy       0.79      0.66      0.72        64
     sadness       0.78      0.61      0.69        77
       shame       0.67      0.50      0.57        70

    accuracy                           0.64       554
   macro avg       0.66      0.63      0.64       554
weighted avg       0.65      0.64      0.64       554



In [54]:
pickle.dump(model, open('emot_lr.pkl','wb'))

In [62]:
pickle.dump(vectorizer, open('emot_vectorizer.pkl','wb'))

In [63]:
vectorizer1 = pickle.load(open('emot_vectorizer.pkl','rb'))

In [64]:
model1 = pickle.load(open('emot_lr.pkl','rb'))
print(model.predict(vectorizer1.transform(["could it be any ?"])))

['fear']


In [45]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='rbf', gamma=0.1)
svc.fit(X_train, y_train)
prediction = svc.predict(X_test)
accuracy_score(y_test,prediction)

0.3935018050541516

In [33]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
rf_predictions = model.predict(X_test)
accuracy_score(y_test,rf_predictions)

e:\test-env\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5613718411552346

In [34]:
model = MultinomialNB()
model.fit(X_train,y_train)
nb_predictions = model.predict(X_test)
accuracy_score(y_test,nb_predictions)

0.5126353790613718

In [35]:
a = vectorizer.transform(["happy "])
model.predict(a)

array(['joy'], dtype='<U7')

In [36]:
import paralleldots
paralleldots.set_api_key("OSIaK6xqY82Dqz5uDAh6ZtAinGLrMjUDcDY7HDuPDFY")
# for single sentence
text="i fear the devil"
response=paralleldots.emotion(text)
print(response)



{'emotion': {'Happy': 0.0733271176, 'Angry': 0.1222725227, 'Fear': 0.1098293354, 'Bored': 0.0367982872, 'Sad': 0.1071247664, 'Excited': 0.5506479707}}


In [37]:
from nltk.corpus import wordnet
synonyms = []
antonyms = []
for syn in wordnet.synsets("happy"): 
    print(syn)
    for l in syn.lemmas(): 
        synonyms.append(l.name()) 
        if l.antonyms(): 
            antonyms.append(l.antonyms()[0].name()) 

Synset('happy.a.01')
Synset('felicitous.s.02')
Synset('glad.s.02')
Synset('happy.s.04')


In [38]:
text = 'not happy'
temp = re.findall(r'\bnot\s\w+\b', text)

In [39]:
for i in temp :
        try :
            for syn in wordnet.synsets(i.split()[1]): 
                for l in syn.lemmas(): 
                    if l.antonyms():
                        print(l.antonyms()[0].name())
                        text = re.sub(r"\bnot\s\w+\b",l.antonyms()[0].name(),text)
        except :
            print(i.split())

unhappy
